## Prerequisites

### 1. Runpod Endpoint

You must have an active Runpod serverless endpoint with PaddleOCR-VL handler deployed.

Set these environment variables:

```bash
export RUNPOD_API_KEY="your_runpod_api_key"
export RUNPOD_ENDPOINT_ID="4nq7w0tw2k8uwk"  # Your endpoint ID
```

Or in `.env` file:

```
RUNPOD_API_KEY=your_runpod_api_key
RUNPOD_ENDPOINT_ID=4nq7w0tw2k8uwk
```

### 2. B2 Credentials (Source)

Credentials for reading PDFs from B2:

```bash
export B2_SOURCE_ACCESS_KEY_ID="your_b2_key_id"
export B2_SOURCE_SECRET_ACCESS_KEY="your_b2_secret"
```

### 3. Optional: Runpod S3 Credentials (Image Extraction)

Only needed if using `--extract-images` to download visualization images:

```bash
export RUNPOD_S3_ACCESS_KEY_ID="user_xxxxx"
export RUNPOD_S3_SECRET_ACCESS_KEY="rps_xxxxx"
export RUNPOD_S3_ENDPOINT_URL="https://s3api-eu-ro-1.runpod.io"
export RUNPOD_S3_REGION="EU-RO-1"
export RUNPOD_NETWORK_VOLUME_ID="u2qv0e9yfd"
```

### 4. Configuration

Configuration is loaded from `config/ocr_config.yaml`. Override specific settings via CLI args or environment variables.

## Basic Usage

### Process a single citekey

```bash
uv run scripts/run_ocr.py --citekeys test_v04 --output data/analytics/ocr
```

### Process multiple citekeys

```bash
uv run scripts/run_ocr.py --citekeys dagz_v01 dagz_v02 dagz_v03 --output data/analytics/ocr
```

### Process from local directory

```bash
uv run scripts/run_ocr.py --input data/sources/test/ --output data/analytics/ocr
```

### Process single local PDF file

```bash
uv run scripts/run_ocr.py --input tests/fixtures/sample_pdfs/test_v04.pdf --output data/analytics/ocr
```

### With image extraction

```bash
uv run scripts/run_ocr.py --citekeys dagz_v01 --output data/analytics/ocr --extract-images
```

### Custom batch size (pages per batch)

```bash
uv run scripts/run_ocr.py --citekeys dagz_v01 dagz_v02 --output data/analytics/ocr --batch-pages 200
```

### Explicit source job ID (for lineage)

```bash
uv run scripts/run_ocr.py --citekeys dagz_v01 dagz_v02 --output data/analytics/ocr --source-job-id 2026-01-01_20-48-14
```

### Auto-detect latest source job

```bash
uv run scripts/run_ocr.py --citekeys dagz_v01 dagz_v02 --output data/analytics/ocr --source-job-id latest
```

### Quiet mode (suppress output)

```bash
uv run scripts/run_ocr.py --citekeys dagz_v01 --output data/analytics/ocr --quiet
```

## How It Works

### Step 1: Discover & Validate Citekeys

The script accepts input in two ways:

**Option A: Explicit citekeys (for B2 source)**
```bash
run_ocr.py --citekeys dagz_v01 dagz_v02 dagz_v03
```

**Option B: Local files (discover citekeys from filenames)**
```bash
run_ocr.py --input data/sources/test/  # Discovers test_v01.pdf, test_v02.pdf, etc.
```

All citekeys are validated against B2 to ensure they were uploaded first via `upload_pdfs.py`.

### Step 2: Auto-detect Source Job IDs (Lineage)

**Default behavior** (if no `--source-job-id` specified):
- Search all job metadata files in `data/sources/job_metadata/`
- For each citekey, find which upload job contains it
- Only trust jobs with `status="completed"` (same safety as upload_pdfs.py)
- Store per-citekey mapping for later

**Example output:**
```
🔍 Auto-detecting source job IDs from citekey metadata...
  dagz_v01: 2026-01-01_20-48-14
  dagz_v02: 2026-01-01_20-48-14
  dagz_v03: 2026-01-02_14-30-22
  Lineage: 3/3 citekey(s) mapped to source job(s)
```

**Why lineage matters:**
- Reproducibility: "Which exact PDF batch produced this OCR?"
- Debugging: "If OCR output is wrong, check the source PDF"
- Audit trail: Complete history from source → upload → OCR

### Step 3: Get File Sizes & Page Counts

**For local files** (from `--input`):
- Reads file size directly from disk
- Uses PyPDF2 to get actual page count (if available)
- Falls back to estimation if PyPDF2 not available

**For B2 files** (from `--citekeys`):
- Queries B2 metadata for file size
- Downloads temp copy to count pages with PyPDF2
- Deletes temp file after counting

### Step 4: Create Batches

Groups PDFs into batches based on **page count** (not file count):

```
📦 Creating batches (target 300 pages per batch)...
   3 batch(es):
     Batch 1:  2 file(s),      100MB (  372 pages)
     Batch 2:  1 file(s),       50MB (  280 pages)
     Batch 3:  1 file(s),       40MB (  195 pages)
```

**Why page-based batching?**
- GPU memory is proportional to pages, not file count
- Balanced batches improve throughput
- Respects `--batch-pages` parameter for tuning

### Step 5: Generate Presigned URLs

For each file, generate a **temporary, limited-access URL** to B2:
- Valid for 12 hours (configurable in `ocr_config.yaml`)
- No B2 credentials embedded
- Unique per file
- Handler receives only URLs, not credentials

### Step 6: Submit to Runpod

For each batch, submit a job with:

```json
{
  "input": {
    "job_id": "2026-01-01_21-45-14",
    "files": [
      {
        "type": "url",
        "url": "https://f000.backblazeb2.com/...",
        "metadata": {
          "citekey": "dagz_v01",
          "job_id": "2026-01-01_21-45-14",
          "batch_idx": 1,
          "source_job_id": "2026-01-01_20-48-14",
          "pipeline_step": "ocr"
        }
      }
    ],
    "pipeline_config": { ... },
    "predict_params": { ... }
  }
}
```

### Step 7: Poll for Completion

Polls Runpod API with exponential backoff:
- Check every 2 seconds (configurable)
- Retry on transient errors (DNS, timeout)
- Fail fast on permanent errors
- Max polling duration: 30 minutes (configurable)

### Step 8: Process Results

When job completes, Runpod returns OCR results:

```json
{
  "results": [
    {
      "metadata": {
        "citekey": "dagz_v01",
        "source_job_id": "2026-01-01_20-48-14",
        "job_id": "2026-01-01_21-45-14"
      },
      "pages": [
        {
          "page_num": 1,
          "text": "...",
          "confidence": 0.92
        }
      ]
    }
  ]
}
```

### Step 9: Save Results & Metadata

Results are saved locally:

```
data/analytics/ocr/
  dagz_v01/
    2026-01-01_21-45-14/
      results.json
      images/             (if --extract-images)
        page_001.png
        page_002.png
        ...
    latest -> 2026-01-01_21-45-14
  job_metadata/
    2026-01-01_21-45-14.json
    latest.json -> 2026-01-01_21-45-14.json
```

Metadata includes:
- Job ID, timestamp
- List of citekeys processed
- Batch info (how many batches, which Runpod job IDs)
- Source job ID (for lineage)
- Pipeline configuration used
- Results summary

## Common Workflows

### Workflow 1: First-time OCR of Uploaded PDFs

```bash
# Just uploaded PDFs via upload_pdfs.py
# Auto-detect source job and process

uv run scripts/run_ocr.py \
  --citekeys dagz_v01 dagz_v02 dagz_v03 \
  --output data/analytics/ocr
```

**What happens:**
- Citekeys validated against B2
- Source job IDs auto-detected from metadata
- Batches created based on page counts
- Results saved to `data/analytics/ocr/<citekey>/2026-01-01_HH-MM-SS/`
- Metadata saved to `data/analytics/ocr/job_metadata/2026-01-01_HH-MM-SS.json`
- `latest` symlinks created for easy access

### Workflow 2: Test with Local PDF Before Full Run

```bash
# Test OCR configuration with a small sample

uv run scripts/run_ocr.py \
  --input tests/fixtures/sample_pdfs/test_v04.pdf \
  --output data/analytics/ocr

# Check results
cat data/analytics/ocr/test_v04/latest/results.json | python -m json.tool
```

### Workflow 3: Large Batch with Custom Page Sizing

```bash
# Process many PDFs, custom batch size for GPU tuning

uv run scripts/run_ocr.py \
  --citekeys dagz_v01 dagz_v02 ... dagz_v50 \
  --output data/analytics/ocr \
  --batch-pages 500  # Larger batches (more GPU memory)
```

### Workflow 4: Extract Visualizations

```bash
# Save PNG visualization images of OCR detection

uv run scripts/run_ocr.py \
  --citekeys dagz_v01 \
  --output data/analytics/ocr \
  --extract-images

# Results include images
ls data/analytics/ocr/dagz_v01/latest/images/
# Output: page_001.png, page_002.png, ...
```

### Workflow 5: Explicit Source Job Tracking

```bash
# Process PDFs from a specific upload batch
# Useful when multiple batches were uploaded

uv run scripts/run_ocr.py \
  --citekeys dagz_v01 dagz_v02 \
  --output data/analytics/ocr \
  --source-job-id 2025-12-31_22-04-05
```

## Configuration

### ocr_config.yaml Structure

Configuration is centralized in `config/ocr_config.yaml`. Key sections:

**B2 Source** (where PDFs are stored):
```yaml
b2_source:
  bucket: cna-sources
  region: us-east-005
  endpoint_url: https://s3.us-east-005.backblazeb2.com
  presigned_url_expiry: 43200  # 12 hours
```

**Batching** (how to group PDFs):
```yaml
batching:
  default_pages_per_batch: 300
  max_pages_per_batch: 1200
```

**Job Submission** (Runpod interaction):
```yaml
job_submission:
  poll_interval: 2           # Check every 2 seconds
  max_poll_duration: 1800    # Max 30 minutes
  max_retries_per_status: 3  # Retry transient errors
```

**PaddleOCR Pipeline Config**:
```yaml
pipeline:
  ocr:
    engine: paddleocr
    languages: [ch, en]
    gpu_id: 0
    batch_size: 32
    model_dir: null  # Use default cached models
```

**PaddleOCR Predict Params** (OCR tuning):
```yaml
predict:
  ocr:
    det_db_thresh: 0.3
    rec_batch_num: 15
    cls_batch_num: 10
    output_format: [json]
```

### Override via CLI

```bash
# Use custom config files
uv run scripts/run_ocr.py \
  --citekeys dagz_v01 \
  --output data/analytics/ocr \
  --pipeline-config my_pipeline.yaml \
  --predict-params my_predict.yaml
```

### Override via Environment Variables

```bash
# Custom B2 bucket
export B2_SOURCE_BUCKET="my-custom-bucket"

# Custom batch size
# (use CLI: --batch-pages 200)

uv run scripts/run_ocr.py --citekeys dagz_v01 --output data/analytics/ocr
```

## Troubleshooting

### Error: Citekey NOT FOUND in B2

**Problem:**
```
✗ dagz_v01 - NOT FOUND in B2 (must be uploaded first via upload_pdfs_to_b2.py)
```

**Solution:** PDF hasn't been uploaded yet. Run `upload_pdfs.py` first:
```bash
uv run scripts/upload_pdfs.py --input data/sources/dagz_v01.pdf
```

### Error: Runpod endpoint not found

**Problem:**
```
✗ Batch 1 submission failed: 404 Client Error: Not Found for url: https://api.runpod.io/v2/4nq7w0tw2k8uwk/run
```

**Solutions:**
1. Check `RUNPOD_ENDPOINT_ID` is correct
2. Verify endpoint is deployed and active on Runpod console
3. Check `RUNPOD_API_KEY` is valid
4. Test endpoint health:
   ```bash
   curl -X GET "https://api.runpod.io/v2/4nq7w0tw2k8uwk/health" \
     -H "Authorization: Bearer $RUNPOD_API_KEY"
   ```

### Error: B2 configuration incomplete

**Problem:**
```
RuntimeError: B2 source configuration incomplete. Required: B2_SOURCE_ACCESS_KEY_ID, B2_SOURCE_SECRET_ACCESS_KEY
```

**Solution:** Set B2 credentials:
```bash
export B2_SOURCE_ACCESS_KEY_ID="your_key_id"
export B2_SOURCE_SECRET_ACCESS_KEY="your_secret"
```

Or add to `.env` file (project root):
```
B2_SOURCE_ACCESS_KEY_ID=your_key_id
B2_SOURCE_SECRET_ACCESS_KEY=your_secret
```

### Warning: Runpod S3 credentials incomplete

**Problem:**
```
WARNING: Runpod S3 credentials incomplete. Image extraction from network volume will be skipped.
```

**Solution** (if you want image extraction):
```bash
export RUNPOD_S3_ACCESS_KEY_ID="user_xxxxx"
export RUNPOD_S3_SECRET_ACCESS_KEY="rps_xxxxx"
export RUNPOD_S3_ENDPOINT_URL="https://s3api-eu-ro-1.runpod.io"
export RUNPOD_S3_REGION="EU-RO-1"
export RUNPOD_NETWORK_VOLUME_ID="u2qv0e9yfd"
```

Otherwise, ignore the warning. Image extraction is optional.

### Error: No source job found for auto-detection

**Problem:**
```
dagz_v01: (not found in source metadata)
```

**Solution:** Either:
1. Re-run `upload_pdfs.py` with this citekey (creates metadata)
2. Explicitly specify source job ID:
   ```bash
   uv run scripts/run_ocr.py --citekeys dagz_v01 --source-job-id 2026-01-01_20-48-14
   ```
3. Or use `--source-job-id latest` to use most recent upload batch

### Error: PyPDF2 not installed

**Problem:**
```
WARNING: Could not get page count for /path/to/pdf.pdf: name 'PyPDF2' is not defined, using estimate
```

**Solution** (optional):
```bash
uv add PyPDF2
```

Without PyPDF2, page counts are estimated from file size (accuracy depends on PDF compression).

## Best Practices

### For Data Workers

1. **Always verify B2 upload first:**
   ```bash
   # Confirm PDFs are in B2 before running OCR
   uv run scripts/upload_pdfs.py --input data/sources/ --dry-run
   ```

2. **Test with small batch:**
   ```bash
   # Process one PDF to verify setup before large batch
   uv run scripts/run_ocr.py --citekeys test_v01 --output data/analytics/ocr
   ```

3. **Monitor Runpod costs:**
   - GPU time is expensive
   - Batch strategically (group similar-sized PDFs)
   - Use reasonable `--batch-pages` values (300-500 is typical)

4. **Document metadata:**
   - Save which job ID you processed
   - Note any configuration changes
   - Link to source upload batch in notes

### For Developers

1. **Test locally first:**
   ```bash
   # Use test fixtures, not production data
   uv run scripts/run_ocr.py --input tests/fixtures/sample_pdfs/ --output data/analytics/ocr
   ```

2. **Check results structure:**
   ```bash
   # Verify output format and metadata
   cat data/analytics/ocr/<citekey>/latest/results.json | python -m json.tool
   cat data/analytics/ocr/job_metadata/latest.json | python -m json.tool
   ```

3. **Monitor lineage integrity:**
   - Verify `source_job_id` is set correctly
   - Cross-check with source metadata
   - Report gaps in lineage tracking

4. **Tune batch size strategically:**
   - Larger batches (500+ pages) = better GPU utilization but more memory
   - Smaller batches (100-200 pages) = faster processing but less efficient
   - Test with your endpoint's GPU spec

### For the Pipeline

1. **Metadata is your audit trail:**
   - Don't delete `data/analytics/ocr/job_metadata/`
   - Metadata enables reproducibility
   - Use it to answer "what was processed when?"

2. **B2 is authoritative:**
   - Citekeys MUST exist in B2 before OCR
   - Validation checks this automatically
   - Prevents orphaned OCR results

3. **Lineage links everything:**
   - OCR job → source upload job → original PDFs
   - Auto-detection reduces manual error
   - Enables complete reproducibility

4. **Results directory structure is standardized:**
   ```
   data/analytics/ocr/
     <citekey>/
       <job_id>/        # Results for this run
         results.json
         images/        # Optional visualization
       latest -> <job_id>  # Easy access to newest
     job_metadata/
       <job_id>.json
       latest.json -> <job_id>.json
   ```
   
   Use this structure in downstream processing.